# Problem 1: Likelihood.



Given a hidden Markov Model $\lambda = (A,B)$ and an observation sequence $O$, determine the likelihood $P(O|\lambda)$ 

* We have an HMM and we know the transition probabilities and the observation likelihoods
* What is the probability of seeing a *specific* sequence of observations?




We start with a simple example

Suppose we knew the weather and wanted to predict how much ice cream Jason will eat.

E.g. for a given hidden state sequence (e.g. *hot hot cold*) compute the output likelihood of *3 1 3*

$$
P(O|Q) = \Pi_{i=1}^TP(o_i|q_i)
$$

So in our example 

$$
P(3\text{ }  1 \text{ } 3 | hot \text{ } hot  \text{ } cold) = P(3|hot) \times P(1 |hot) \times P(3 |cold )
$$

Which we know


In [4]:
from coefficients import B, PI,A
from markov import hot, cold
from math import prod

In [5]:
print((B[hot][3]) * (B[hot][1]) * B[cold][3])

1/125


Now let's sum over all possible weather sequences *weighted by their probability*

First, the joint probability of being in a particular weather sequence $Q$ and generating a particular sequence $O$ of ice-cream events.

$$
P(O,Q)  = P(O|Q) \times P(Q) = \Pi_{i=1}^T P(o_i | q_i) \times \Pi_{i=1}^T P( q_i | q_{i-1})
$$

So given sequence hot, hot, cold, 

What is `P(3 1 3 | hot hot cold)` ?

In [6]:
sequence = [hot,hot,cold]
observations = [3,1,3]

$$
P(3 \space 1 \space 3 , hot\space hot \space cold)
= 
P(hot | start ) \times P(hot | hot ) \times P(cold | hot)
\times 
P(3 | hot ) \times P(1 | hot) \times P( 3 | cold )
$$

In [7]:
p = PI[hot] * A[hot][hot] * A[hot][cold] * B[hot][3] * B[hot][1] * B[cold][3]
p

Fraction(24, 15625)

More generally:

In [8]:
from algorithms import p_observations_given_states, p_states

In [9]:
p_states(PI,A,sequence)

Fraction(24, 125)

In [10]:
p_observations_given_states(B, sequence, observations)

Fraction(1, 125)

In [11]:
p = p_states(PI,A,sequence) * p_observations_given_states(B, sequence, observations)
print(p)
print(float(p))

24/15625
0.001536


And now we just need to sum over all possible sequences

$$
P(O) = \Sigma_{Q} P(O,Q) = \Sigma_Q P(O|Q) P(Q)
$$

In [12]:
weather_sequences = [
    (s1, s2, s3) for s1 in [hot, cold] for s2 in [hot, cold] for s3 in [hot, cold]
]
weather_sequences


[(hot, hot, hot),
 (hot, hot, cold),
 (hot, cold, hot),
 (hot, cold, cold),
 (cold, hot, hot),
 (cold, hot, cold),
 (cold, cold, hot),
 (cold, cold, cold)]

In [13]:
for s1, s2, s3 in weather_sequences:
    p_o = (B[s1][3]) * (B[s2][1]) * (B[s3][3])
    print(f"Probability of seeing (3,1,3) after weather ({s1}, {s2}, {s3}) is {p_o}")
    print(f"Probability of seeing weather ({s1}, {s2}, {s3}) is {p_states(PI,A, [s1,s2,s3])}")

    assert p_o==(p_observations_given_states(B,[s1,s2,s3],[3,1,3]))


Probability of seeing (3,1,3) after weather (hot, hot, hot) is 4/125
Probability of seeing weather (hot, hot, hot) is 36/125
Probability of seeing (3,1,3) after weather (hot, hot, cold) is 1/125
Probability of seeing weather (hot, hot, cold) is 24/125
Probability of seeing (3,1,3) after weather (hot, cold, hot) is 2/25
Probability of seeing weather (hot, cold, hot) is 4/25
Probability of seeing (3,1,3) after weather (hot, cold, cold) is 1/50
Probability of seeing weather (hot, cold, cold) is 4/25
Probability of seeing (3,1,3) after weather (cold, hot, hot) is 1/125
Probability of seeing weather (cold, hot, hot) is 3/50
Probability of seeing (3,1,3) after weather (cold, hot, cold) is 1/500
Probability of seeing weather (cold, hot, cold) is 1/25
Probability of seeing (3,1,3) after weather (cold, cold, hot) is 1/50
Probability of seeing weather (cold, cold, hot) is 1/20
Probability of seeing (3,1,3) after weather (cold, cold, cold) is 1/200
Probability of seeing weather (cold, cold, cold)

In [14]:
p_o = sum([
    p_observations_given_states(B,states, observations ) * p_states(PI,A,states )

    for 
    states in weather_sequences
])

print(p_o)

14281/500000


In [15]:
print(f"""

    Overally likelihood of seeing observations {observations} is {p_o:.4f}
""")



    Overally likelihood of seeing observations [3, 1, 3] is 0.0286





### Computational cost

Unfortunately brute-forcing over all possible sequences can become computationally infeasible, it's something like $O(N^i)$

---

## Forward Algorithm 

This is more computationally tractable than the brute-force approach, and works by caching intermediate values.




We compute this with 'cells'. Each cell of the forward algorithm trellis $\alpha_t(j)$ represents the probability of being in state $j$ after seeing the first $t$ observations, given the automaton $\lambda$.

The value of each cell $\alpha_t(j)$ is computed by summing over the probabilities of every path that could lead us to this cell.

Each cell expresses the following probability:

$$

\alpha_t(j) = P(o_1, o_2,...,o_t,q_t = j|\lambda)

$$



i.e. at time step $t$, $\alpha_t(j)$ is the probability of having seen the full sequence of observations 

$$
    \alpha_t(j) = \Sigma_{i=1}^N \alpha_{t-1}(i) a_{ij} b_j(o_t)
$$

$P(H|start) \times P(2|H) = 0.32 $ $

In [16]:
from markov import hot, cold, Temperature
from fractions import Fraction
from itertools import islice 
from typing import Iterable 

## Use the same weights as before

In [17]:
from coefficients import A,B,PI

See Figure A.5

In [18]:
alpha_1_1 = PI[cold] * B[cold][3]
alpha_1_2 = PI[hot] * B[hot][3]

alpha_2_1 = (alpha_1_2 * A[hot][cold] + alpha_1_1 * A[cold][cold]) * B[cold][1]
alpha_2_2 = (alpha_1_2 * A[hot][hot] + alpha_1_1 * A[cold][hot]) * B[hot][1]


assert float(alpha_1_1) == 0.02, alpha_1_1
assert float(alpha_1_2) == 0.32, alpha_1_2
assert float(alpha_2_1) == 0.069, alpha_2_1
assert float(alpha_2_2) == 0.0404, float(alpha_2_2)



Note that this is a recursive algorithm: 
$$
    \alpha_t(j) = \Sigma_{i=1}^N \alpha_{t-1}(i) a_{ij} b_j(o_t)
$$

- (A.12)

Because $\alpha_t$ depends on $\alpha_{t-1}$

We can implement this in Python as a generator

In [19]:
from algorithms import forward

In [20]:
observations = [3, 1, 3]
alphas = list(forward(PI,A,B,observations))
alphas 

[{cold: Fraction(1, 50), hot: Fraction(8, 25)},
 {cold: Fraction(69, 1000), hot: Fraction(101, 2500)},
 {cold: Fraction(2533, 500000), hot: Fraction(2937, 125000)}]

In [21]:
[
    {key:float(value) for key, value in alpha.items() }
    for alpha in alphas
]

[{cold: 0.02, hot: 0.32},
 {cold: 0.069, hot: 0.0404},
 {cold: 0.005066, hot: 0.023496}]

## Finally


$$
P(O | \lambda) = \Sigma_{i=1}^N \alpha_T(i)
$$

In [22]:
for (observation, alpha) in zip(observations, alphas):
    print(f"""
        Observation: {observation} ice creams eaten.
        P(hot) = {float(alpha[hot]):.4f}, P(cold) = {float(alpha[cold]):.4f}
        P(O|λ)  = {float(sum(alpha.values())):.4f}
""")


        Observation: 3 ice creams eaten.
        P(hot) = 0.3200, P(cold) = 0.0200
        P(O|λ)  = 0.3400


        Observation: 1 ice creams eaten.
        P(hot) = 0.0404, P(cold) = 0.0690
        P(O|λ)  = 0.1094


        Observation: 3 ice creams eaten.
        P(hot) = 0.0235, P(cold) = 0.0051
        P(O|λ)  = 0.0286



Which is in agreement with the brute force approach.